In [2]:
import pandas as pd
import numpy as np

In [3]:
all_dataset = pd.read_csv("data/arrhythmia.data", header=None)

In [4]:
chosen_column = [0] + [i for i in range(3, 15)] + [all_dataset.columns[-1]]
#chosen_column = [0] + [i for i in range(3, 13)] + [14] + [all_dataset.columns[-1]]
max_trash = 10

In [5]:
def clean_data(data):
    for col in data.columns:
        trash_row = (data[col] == "?")
        good_row = (data[col] != "?")
        if sum(trash_row) < max_trash:
            data = data[good_row]
        else:
            aver =  pd.to_numeric(data[good_row][col]).mean()
            data.loc[trash_row, col] = aver
        data[col] = pd.to_numeric(data[col])
        data[col] -= data[col].min()
    return data

In [6]:
#use standart skleaarn func or use your own
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

#also could be a good idea to do feature scaling
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#scaler.fit(X_train)

#X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)
#see more on https://stackabuse.com/k-nearest-neighbors-algorithm-in-python-and-scikit-learn/

In [7]:
def split_train_test(df):
    margin = int(0.8 * len(df))
    df = df.sample(frac=1)
    train = df.iloc[:margin]
    test = df.iloc[margin:]
    return train, test

In [8]:
#3,4 -> 3; 5,6 -> 4; 9, 10 -> 5; other -> 6

def reduce_class(y):
    y[(y == 2) | (y == 3)] = 2
    y[(y == 4) | (y == 5)] = 3
    y[(y == 8) | (y == 9)] = 4
    y[y > 5] = 5
    return y    

In [9]:
#3,4 -> 3; 5,6 -> 4; 9, 10 -> 5; other -> 6

def reduce_class(y):
    y[y > 0] = 1
    return y    

In [10]:
def from_data_to_xy(data):
    x = np.array(data[data.columns[:-1]], dtype=float)
    y = np.array(data[data.columns[-1]], dtype=float)
    return x, y

In [11]:
def from_data_to_xy_reduce(data):
    x = np.array(data[data.columns[:-1]], dtype=float)
    y = np.array(data[data.columns[-1]], dtype=float)
    y = reduce_class(y)
    return x, y

## class = 13
|Method | test on all data | cross validation|
-------------------------------------
|GaussianNB | 0.658  | 0.5 |
---------------------------------------
|ComplementNB | 0.536 | 0.48 |
--------------------------------------------
|MultinomialNB | 0.47 | 0.43 |
-----------------------------------------


### if delete almost emply column № 13(14), results become better
-----------------------------------------------------
|Method | test on all data | cross validation|
-------------------------------------------------------------
|GaussianNB | 0.6832  | 0.6|
|ComplementNB | 0.5226 | 0.49|
|MultinomialNB | 0.5  | 0.45|

for class = 6
GaussianNB = 0.58
ComplementNB = 0.536 || 0.48
MultinomialNB = 0.47  || 0.43

# n class = 2
GaussianNB = 0.72
ComplementNB = 0.628
MultinomialNB = 0.625

In [12]:
from sklearn import naive_bayes

def bayes_classifier(which='gaussian'):
    classifier = {
        'gaussian' : naive_bayes.GaussianNB(),
        'complement' : naive_bayes.ComplementNB(),
        'multinomial' : naive_bayes.MultinomialNB()
    }
    clf = classifier[which]
    return clf    

In [13]:
def check_accuracy(data, classifier, num_iter=1000, d_to_xy=from_data_to_xy_reduce):
    accuracy = []
    for i in range(num_iter):
        train_data, test_data = split_train_test(data)
        train_x, train_y = d_to_xy(train_data)
        test_x, test_y = d_to_xy(test_data)
        clf = classifier.fit(train_x, train_y)
        accuracy.append(sum(clf.predict(test_x) == test_y)  / test_y.shape[0])
    #print(test_y)
    return accuracy

In [14]:
dataset = all_dataset[chosen_column]
clean_dataset = clean_data(dataset)

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [ ]:
#which = 'multinomial'
#which = 'gaussian'
which = 'complement'

ac = check_accuracy(clean_dataset, bayes_classifier(which), 1000)
print(np.mean(ac))

knn:
5     0.5802134831460674
6     0.5839887640449437
7     0.5853932584269662
8     0.5837415730337078
9     0.5767640449438202
10     0.5775393258426966
11     0.569752808988764
12     0.5705056179775281
13     0.5678651685393259

knn with all columns:
5     0.5801235955056179
6     0.584
7     0.5836179775280897
8     0.5845842696629213
9     0.5774494382022471
10     0.574876404494382
11     0.5708988764044943
12     0.570820224719101
13     0.5677303370786517


knn for  k =6
1     0.5240449438202247
2     0.5580898876404494
3     0.5666292134831461
4     0.5801123595505617
5     0.5835955056179776
6     0.582808988764045
7     0.5811235955056178
8     0.5842696629213484
9     0.5857303370786517

knn for k = 2
1     0.6507865168539325
2     0.6473033707865169
3     0.6506741573033709
4     0.656741573033708
5     0.6574157303370788
6     0.662808988764045
7     0.6735955056179777
8     0.6610112359550563
9     0.666629213483146

In [101]:
from sklearn.neighbors import KNeighborsClassifier
for i in range(1,10):
    knn_classifier = KNeighborsClassifier(n_neighbors=i)
    ac = check_accuracy(clean_dataset, knn_classifier, 100)
    print(i, "   ", np.mean(ac))

1     0.6507865168539325
2     0.6473033707865169
3     0.6506741573033709
4     0.656741573033708
5     0.6574157303370788
6     0.662808988764045
7     0.6735955056179777
8     0.6610112359550563
9     0.666629213483146


            ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’
    with || 
    without || 0.582 || 0.5346 || 0.5514 || 0.5437

svc linear with k = 6
0.56
svc linear with k = 2
c=1 || 0.6786516853932584
c=0.7 || 0.66
c=1.3 || 0.67

In [16]:
from sklearn.svm import SVC
clf = SVC(kernel='linear', C=1.3)
ac = check_accuracy(clean_dataset, clf, 5)
print(np.mean(ac))

0.6786516853932584


In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='poly', gamma='auto')
ac = check_accuracy(clean_dataset, clf, 5)
print(np.mean(ac))

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [44]:
from sklearn.svm import SVC
clf = SVC(kernel='rbf')
ac = check_accuracy(clean_dataset, clf, 100)
print(np.mean(ac))

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

0.5514606741573034


/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [45]:
from sklearn.svm import SVC
clf = SVC(kernel='sigmoid')
ac = check_accuracy(clean_dataset, clf, 100)
print(np.mean(ac))

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

0.5476404494382023


/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [39]:
#chosen_factor_column = [0, 1] + [i for i in range(3, 13)] + [14] + [all_dataset.columns[-1]]
chosen_factor_column = [0, 1] + [i for i in range(3, 15)] + [all_dataset.columns[-1]]

In [30]:
new_dataset = all_dataset[chosen_factor_column]
clean_new_dataset = clean_data(new_dataset)

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [55]:
data1 = clean_new_dataset[clean_new_dataset[1] > 0.5][chosen_column]
data2 = clean_new_dataset[clean_new_dataset[1] < 0.5][chosen_column]

In [56]:
print(data2)

     0    3    4    5    6    7    8    9    10     11   12   14   279
0     75   74   36  193  139   66  121  156  190  234.0  133   19    7
2     54   89   83  163  154   77  102  268  211  240.0  201   31    9
3     55   88   45  202  148   71  143  200  188  165.0  155   27    0
5     13   45   45  167   89   66   91  279  243  222.0  223   40   13
8     44   50   29  118  122   52   63  233  246  248.0  201   20    0
..   ...  ...  ...  ...  ...  ...  ...  ...  ...    ...  ...  ...  ...
440   37   66   33  153  157   64   89  239  225  228.0  193   11    0
444   29   57   26  143   93  110   74  196  204  202.0  160   34    0
445   45   69   36  134  144   52   83  263  245  201.0  215   26    0
448   37   79   45  137  129   93   73  258  243  222.0  214   29    9
449   36   62   53  176  133   86  116   87  158  109.0   65   40    1

[198 rows x 13 columns]


In [57]:
which = 'multinomial'
#which = 'gaussian'
#which = 'multinomial'

ac = check_accuracy(data1, bayes_classifier(which), 1000)
print(np.mean(ac))

0.4659795918367347


In [58]:
#which = 'multinomial'
which = 'gaussian'
#which = 'multinomial'

ac = check_accuracy(data1, bayes_classifier(which), 1000)
print(np.mean(ac))

0.6766122448979591


In [35]:
#which = 'multinomial'
#which = 'gaussian'
which = 'complement'

ac = check_accuracy(data1, bayes_classifier(which), 1000)
print(np.mean(ac))

0.5624489795918367


for data1
(with / without)
'multinomial'  0.4779183673469388 || 0.46075510204081627
'gaussian' 0.5273061224489797 || 0.6748367346938776
'complement' 0.5630816326530612 || 0.5624489795918367

In [36]:
which = 'multinomial'
#which = 'gaussian'
#which = 'multinomial'

ac = check_accuracy(data2, bayes_classifier(which), 1000)
print(np.mean(ac))

0.33677500000000005


In [37]:
#which = 'multinomial'
which = 'gaussian'
#which = 'multinomial'

ac = check_accuracy(data2, bayes_classifier(which), 1000)
print(np.mean(ac))

0.42795


In [38]:
#which = 'multinomial'
#which = 'gaussian'
which = 'complement'

ac = check_accuracy(data2, bayes_classifier(which), 1000)
print(np.mean(ac))

0.33185000000000003


for data2
(with / without)
'multinomial'  0.354375 || 0.33677500000000005
'gaussian' 0.305225 || 0.42795
'complement' 0.34082499999999993 || 0.33185000000000003

data1 knn

5     0.6587142857142857
6     0.6606938775510204
7     0.6598163265306123
8     0.6574285714285714
9     0.6592244897959183
10     0.6541428571428571
11     0.6573265306122449
12     0.6537142857142857
13     0.6521224489795918

In [40]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=13)

for i in range(5,14):
    knn_classifier = KNeighborsClassifier(n_neighbors=i)
    ac = check_accuracy(data1, knn_classifier, 1000)
    print(i, "   ", np.mean(ac))

5     0.6587142857142857
6     0.6606938775510204
7     0.6598163265306123
8     0.6574285714285714
9     0.6592244897959183
10     0.6541428571428571
11     0.6573265306122449
12     0.6537142857142857
13     0.6521224489795918


data2

knn
5     0.44755
6     0.443775
7     0.441775
8     0.43277499999999997
9     0.43129999999999996
10     0.42892499999999995
11     0.43079999999999996
12     0.42047500000000004
13     0.4161


In [41]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=13)

for i in range(5,14):
    knn_classifier = KNeighborsClassifier(n_neighbors=i)
    ac = check_accuracy(data2, knn_classifier, 1000)
    print(i, "   ", np.mean(ac))

5     0.44755
6     0.443775
7     0.441775
8     0.43277499999999997
9     0.43129999999999996
10     0.42892499999999995
11     0.43079999999999996
12     0.42047500000000004
13     0.4161


    data1  ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed'
    with || 0.682 || 0.614 || 0.6516 || 0.66

In [46]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')
ac = check_accuracy(data1, clf, 100)
print(np.mean(ac))

0.6828571428571426


In [47]:
from sklearn.svm import SVC
clf = SVC(kernel='poly')
ac = check_accuracy(data1, clf, 100)
print(np.mean(ac))

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

0.6140816326530613


In [48]:
from sklearn.svm import SVC
clf = SVC(kernel='rbf')
ac = check_accuracy(data1, clf, 100)
print(np.mean(ac))

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

0.6516326530612245


/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

In [49]:
from sklearn.svm import SVC
clf = SVC(kernel='sigmoid')
ac = check_accuracy(data1, clf, 100)
print(np.mean(ac))

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

0.6606122448979592


/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

In [ ]:
    data2 ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed'
    with || 0.461 || 0.379 || 0.4197 || 0.4225

In [50]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')
ac = check_accuracy(data2, clf, 100)
print(np.mean(ac))

0.4617499999999999


In [51]:
from sklearn.svm import SVC
clf = SVC(kernel='poly')
ac = check_accuracy(data2, clf, 100)
print(np.mean(ac))

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

0.379


In [52]:
from sklearn.svm import SVC
clf = SVC(kernel='rbf')
ac = check_accuracy(data2, clf, 100)
print(np.mean(ac))

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

0.41975


/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

In [53]:
from sklearn.svm import SVC
clf = SVC(kernel='sigmoid')
ac = check_accuracy(data2, clf, 100)
print(np.mean(ac))

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

0.4225


/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/Statistical learning/venv/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/asta/St

In [148]:
import collections

_, y_ = from_data_to_xy(clean_dataset)

print(collections.Counter(y))

print(np.unique(y))

Counter({0.0: 243, 9.0: 50, 1.0: 39, 5.0: 25, 15.0: 21, 2.0: 15, 3.0: 14, 4.0: 13, 8.0: 9, 14.0: 5, 13.0: 4, 7.0: 2, 6.0: 2})
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 13. 14. 15.]


спробувати класи обьеднати деяки

спробувати вибрати 